In [101]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from tensorflow.keras.utils import to_categorical
import re

In [121]:
DATA_DIR = "new_data_2.csv"
FORMATTED_DATA_SAVE_DIR = "formatted_data3.csv"

In [122]:
geolocator = Nominatim(user_agent="ENSE411")

In [123]:
df = pd.read_csv(DATA_DIR)
df.columns =['address','price','beds','bath','square','build','year','style']

In [124]:
new_df = pd.DataFrame(index=range(0,len(df)), columns=range(9))

#### Asign Longitude and Latittude using the geolocator

In [125]:
none_count = 0
for i, row in df.iterrows():
    location = ",".join((row["address"]).split(",", 2)[:2])
    #print(location)
    loc = geolocator.geocode(location)
    if loc:
        lat = loc.latitude
        long = loc.longitude
        
        # create entry in dataframe
        new_df.at[i, 'address'] = row["address"]
        new_df.at[i, 'latitude'] = lat
        new_df.at[i, 'longitude'] = long
        new_df.at[i, "price"] = re.sub(r"\D+", "", row["price"])
        new_df.at[i, 'beds'] = row["beds"]
        new_df.at[i, 'bath'] = row["bath"]
        new_df.at[i, "square"] = re.sub(r"\D+", "", row["square"])
        new_df.at[i, 'build'] = row["build"]
        new_df.at[i, 'year'] = row["year"]
        new_df.at[i, 'style'] = row["style"]
        
        
        
    else:
        none_count += 1
        
print(none_count)

14


In [132]:
new_df_copy = new_df[['address', 'latitude', 'longitude', 'price','beds','bath','square', 'build','year','style']].copy()
new_df_copy.shape
new_df_copy

,address,latitude,longitude,price,beds,bath,square,build,year,style
0,"1139 Rae STREET, Regina, Saskatchewan, S4T 2B9",50.460656,-104.620734,36000,2.0,1.0,840,Residential,1951.0,Freehold
1,"4012 Castle ROAD, Regina, Saskatchewan, S4S 6A4",50.410873,-104.599646,145000,3.0,2.0,1034,Condominium,1973.0,"2 Storey, Row/Townhouse"
2,"138 Trudelle CRESCENT, Regina, Saskatchewan, S...",50.470319,-104.672410,259900,3.0,1.0,850,Residential,1973.0,Freehold
3,"1621 Dakota DRIVE 205, Regina, Saskatchewan, S...",50.452113,-104.529413,259000,2.0,2.0,1130,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
4,"5665 Cederholm AVENUE, Regina, Saskatchewan, S...",50.413116,-104.662669,369900,2.0,3.0,1358,Residential,2014.0,Freehold
...,...,...,...,...,...,...,...,...,...,...
153,"2311 Windsor Park ROAD 209, Regina, Saskatchew...",50.442713,-104.527328,198950,1.0,1.0,829,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,"210 Toronto STREET N, Regina, Saskatchewan, S4...",50.479187,-104.599484,289900,3.0,2.0,1285,Residential,1966.0,Freehold
156,"2315 7th AVENUE E, Regina, Saskatchewan, S4N 4T1",50.458153,-104.550317,270000,3.0,2.0,1146,Residential,1974.0,Freehold


In [133]:
# get rid of every bad data point (NaN)
new_df_copy = new_df_copy.dropna()

In [134]:
new_df_copy

,address,latitude,longitude,price,beds,bath,square,build,year,style
0,"1139 Rae STREET, Regina, Saskatchewan, S4T 2B9",50.460656,-104.620734,36000,2.0,1.0,840,Residential,1951.0,Freehold
1,"4012 Castle ROAD, Regina, Saskatchewan, S4S 6A4",50.410873,-104.599646,145000,3.0,2.0,1034,Condominium,1973.0,"2 Storey, Row/Townhouse"
2,"138 Trudelle CRESCENT, Regina, Saskatchewan, S...",50.470319,-104.672410,259900,3.0,1.0,850,Residential,1973.0,Freehold
3,"1621 Dakota DRIVE 205, Regina, Saskatchewan, S...",50.452113,-104.529413,259000,2.0,2.0,1130,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
4,"5665 Cederholm AVENUE, Regina, Saskatchewan, S...",50.413116,-104.662669,369900,2.0,3.0,1358,Residential,2014.0,Freehold
...,...,...,...,...,...,...,...,...,...,...
151,"4815 Sherwood DRIVE, Regina, Saskatchewan, S4R...",50.476754,-104.650052,349900,4.0,2.0,1020,Residential,1961.0,Freehold
153,"2311 Windsor Park ROAD 209, Regina, Saskatchew...",50.442713,-104.527328,198950,1.0,1.0,829,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
155,"210 Toronto STREET N, Regina, Saskatchewan, S4...",50.479187,-104.599484,289900,3.0,2.0,1285,Residential,1966.0,Freehold
156,"2315 7th AVENUE E, Regina, Saskatchewan, S4N 4T1",50.458153,-104.550317,270000,3.0,2.0,1146,Residential,1974.0,Freehold


In [135]:
new_df_copy = new_df_copy.reset_index()

In [136]:
new_df_copy.drop('index', axis=1, inplace=True)
new_df_copy

,address,latitude,longitude,price,beds,bath,square,build,year,style
0,"1139 Rae STREET, Regina, Saskatchewan, S4T 2B9",50.460656,-104.620734,36000,2.0,1.0,840,Residential,1951.0,Freehold
1,"4012 Castle ROAD, Regina, Saskatchewan, S4S 6A4",50.410873,-104.599646,145000,3.0,2.0,1034,Condominium,1973.0,"2 Storey, Row/Townhouse"
2,"138 Trudelle CRESCENT, Regina, Saskatchewan, S...",50.470319,-104.672410,259900,3.0,1.0,850,Residential,1973.0,Freehold
3,"1621 Dakota DRIVE 205, Regina, Saskatchewan, S...",50.452113,-104.529413,259000,2.0,2.0,1130,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
4,"5665 Cederholm AVENUE, Regina, Saskatchewan, S...",50.413116,-104.662669,369900,2.0,3.0,1358,Residential,2014.0,Freehold
...,...,...,...,...,...,...,...,...,...,...
139,"4815 Sherwood DRIVE, Regina, Saskatchewan, S4R...",50.476754,-104.650052,349900,4.0,2.0,1020,Residential,1961.0,Freehold
140,"2311 Windsor Park ROAD 209, Regina, Saskatchew...",50.442713,-104.527328,198950,1.0,1.0,829,Condominium,2012.0,"Single-Level, Low-Rise (3 floors and under)"
141,"210 Toronto STREET N, Regina, Saskatchewan, S4...",50.479187,-104.599484,289900,3.0,2.0,1285,Residential,1966.0,Freehold
142,"2315 7th AVENUE E, Regina, Saskatchewan, S4N 4T1",50.458153,-104.550317,270000,3.0,2.0,1146,Residential,1974.0,Freehold


In [137]:
df_cont = new_df_copy[['latitude','longitude','price', 'beds','bath', 'square', 'year']].copy()

In [138]:
df_cont

,latitude,longitude,price,beds,bath,square,year
0,50.460656,-104.620734,36000,2.0,1.0,840,1951.0
1,50.410873,-104.599646,145000,3.0,2.0,1034,1973.0
2,50.470319,-104.672410,259900,3.0,1.0,850,1973.0
3,50.452113,-104.529413,259000,2.0,2.0,1130,2012.0
4,50.413116,-104.662669,369900,2.0,3.0,1358,2014.0
...,...,...,...,...,...,...,...
139,50.476754,-104.650052,349900,4.0,2.0,1020,1961.0
140,50.442713,-104.527328,198950,1.0,1.0,829,2012.0
141,50.479187,-104.599484,289900,3.0,2.0,1285,1966.0
142,50.458153,-104.550317,270000,3.0,2.0,1146,1974.0


### One-Hot Encode "build" Feature

In [139]:
builds = new_df_copy['build'].unique()
len(builds)

2

In [140]:
# build dictionary where we enumerate target names, each name has the vales of the index
target_dict = {n:i for i, n in enumerate(builds)}
target_dict

{'Residential': 0, 'Condominium': 1}

In [141]:
builds_num = new_df_copy['build'].map(target_dict)
builds_num

0      0
1      1
2      0
3      1
4      0
      ..
139    0
140    1
141    0
142    0
143    0
Name: build, Length: 144, dtype: int64

In [142]:
build_hotencoded = to_categorical(builds_num)
build_hotencoded

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [143]:
build_df = pd.DataFrame(build_hotencoded, columns = ['Residential','Condominium'])

In [144]:
df = df_cont.join(build_df)
df

,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,50.460656,-104.620734,36000,2.0,1.0,840,1951.0,1.0,0.0
1,50.410873,-104.599646,145000,3.0,2.0,1034,1973.0,0.0,1.0
2,50.470319,-104.672410,259900,3.0,1.0,850,1973.0,1.0,0.0
3,50.452113,-104.529413,259000,2.0,2.0,1130,2012.0,0.0,1.0
4,50.413116,-104.662669,369900,2.0,3.0,1358,2014.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
139,50.476754,-104.650052,349900,4.0,2.0,1020,1961.0,1.0,0.0
140,50.442713,-104.527328,198950,1.0,1.0,829,2012.0,0.0,1.0
141,50.479187,-104.599484,289900,3.0,2.0,1285,1966.0,1.0,0.0
142,50.458153,-104.550317,270000,3.0,2.0,1146,1974.0,1.0,0.0


In [145]:
# Save the dataset as a new csv
df.to_csv(FORMATTED_DATA_SAVE_DIR)

### Combine all batches into the full dataset

In [151]:
df1 = pd.read_csv("formatted_data1.csv", index_col=0)
df1

,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,50.421741,-104.618567,499900,4.0,3.0,1720,1952.0,1.0,0.0
1,50.453769,-104.547280,259900,3.0,2.0,923,1974.0,1.0,0.0
2,50.415279,-104.649451,439900,3.0,4.0,1406,2011.0,0.0,1.0
3,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
4,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
479,50.404034,-104.656729,259900,2.0,2.0,947,2013.0,0.0,1.0
480,50.490740,-104.685706,129900,2.0,1.0,823,1980.0,0.0,1.0
481,50.402380,-104.649176,179950,2.0,1.0,753,2010.0,0.0,1.0
482,50.449838,-104.593449,54900,2.0,1.0,520,1914.0,1.0,0.0


In [152]:
df2 = pd.read_csv("formatted_data2.csv", index_col=0)
df2

,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,50.460635,-104.622808,36000,2.0,1.0,631,1921.0,1.0,0.0
1,50.420778,-104.527582,435000,4.0,3.0,1284,1993.0,1.0,0.0
2,50.450714,-104.567510,239900,4.0,4.0,1593,1964.0,1.0,0.0
3,50.467284,-104.626520,29800,2.0,1.0,600,1941.0,1.0,0.0
4,50.461521,-104.638724,77900,1.0,1.0,528,1946.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
91,50.479240,-104.703274,689900,4.0,3.0,1902,1987.0,1.0,0.0
92,50.492564,-104.686345,298000,5.0,2.0,1140,1979.0,1.0,0.0
93,50.442805,-104.622162,219900,1.0,2.0,1526,1911.0,1.0,0.0
94,50.410946,-104.520632,1349000,4.0,4.0,2720,2013.0,1.0,0.0


In [153]:
df3 = pd.read_csv("formatted_data3.csv", index_col=0)
df3

,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,50.460656,-104.620734,36000,2.0,1.0,840,1951.0,1.0,0.0
1,50.410873,-104.599646,145000,3.0,2.0,1034,1973.0,0.0,1.0
2,50.470319,-104.672410,259900,3.0,1.0,850,1973.0,1.0,0.0
3,50.452113,-104.529413,259000,2.0,2.0,1130,2012.0,0.0,1.0
4,50.413116,-104.662669,369900,2.0,3.0,1358,2014.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
139,50.476754,-104.650052,349900,4.0,2.0,1020,1961.0,1.0,0.0
140,50.442713,-104.527328,198950,1.0,1.0,829,2012.0,0.0,1.0
141,50.479187,-104.599484,289900,3.0,2.0,1285,1966.0,1.0,0.0
142,50.458153,-104.550317,270000,3.0,2.0,1146,1974.0,1.0,0.0


In [154]:
dataset = pd.concat([df1,df2,df3])
dataset

,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,50.421741,-104.618567,499900,4.0,3.0,1720,1952.0,1.0,0.0
1,50.453769,-104.547280,259900,3.0,2.0,923,1974.0,1.0,0.0
2,50.415279,-104.649451,439900,3.0,4.0,1406,2011.0,0.0,1.0
3,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
4,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
139,50.476754,-104.650052,349900,4.0,2.0,1020,1961.0,1.0,0.0
140,50.442713,-104.527328,198950,1.0,1.0,829,2012.0,0.0,1.0
141,50.479187,-104.599484,289900,3.0,2.0,1285,1966.0,1.0,0.0
142,50.458153,-104.550317,270000,3.0,2.0,1146,1974.0,1.0,0.0


In [155]:
dataset = dataset.reset_index()
dataset

,index,latitude,longitude,price,beds,bath,square,year,Residential,Condominium
0,0,50.421741,-104.618567,499900,4.0,3.0,1720,1952.0,1.0,0.0
1,1,50.453769,-104.547280,259900,3.0,2.0,923,1974.0,1.0,0.0
2,2,50.415279,-104.649451,439900,3.0,4.0,1406,2011.0,0.0,1.0
3,3,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
4,4,50.450378,-104.599131,134900,3.0,2.0,1105,2006.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
719,139,50.476754,-104.650052,349900,4.0,2.0,1020,1961.0,1.0,0.0
720,140,50.442713,-104.527328,198950,1.0,1.0,829,2012.0,0.0,1.0
721,141,50.479187,-104.599484,289900,3.0,2.0,1285,1966.0,1.0,0.0
722,142,50.458153,-104.550317,270000,3.0,2.0,1146,1974.0,1.0,0.0


In [156]:
# Save the full dataset
dataset.to_csv("dataset.csv")